In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from glob import glob
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import matplotlib.pylab as plt
import os
import shutil
import cv2
from sklearn import tree
from sklearn.metrics import confusion_matrix
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:



# Paths for the dataset
train_path = "/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train"
validation_path = "/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid"
output_train_path = "/kaggle/working/reduced_dataset/train"
output_validation_path = "/kaggle/working/reduced_dataset/valid"

# # Number of images per class
train_samples = 1000
validation_samples = 300

# # Function to sample and copy images
def create_subset(input_path, output_path, samples_per_class):
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    for class_name in os.listdir(input_path):
        class_dir = os.path.join(input_path, class_name)
        if os.path.isdir(class_dir):
            images = os.listdir(class_dir)
            
            selected_images = images[:samples_per_class]
            class_output_dir = os.path.join(output_path, class_name)
            os.makedirs(class_output_dir, exist_ok=True)
            for image in selected_images:
                src = os.path.join(class_dir, image)
                dst = os.path.join(class_output_dir, image)
                shutil.copy(src, dst)
            print(f"Processed class '{class_name}' with {samples_per_class} images.")

# Create subsets for training and validation
create_subset(train_path, output_train_path, train_samples)
create_subset(validation_path, output_validation_path, validation_samples)
create_subset("/kaggle/input/new-plant-diseases-dataset/test","/kaggle/working/reduced_dataset/test",33)
print("Subset creation completed.")


Processed class 'Tomato___Late_blight' with 1000 images.
Processed class 'Tomato___healthy' with 1000 images.
Processed class 'Grape___healthy' with 1000 images.
Processed class 'Orange___Haunglongbing_(Citrus_greening)' with 1000 images.
Processed class 'Soybean___healthy' with 1000 images.
Processed class 'Squash___Powdery_mildew' with 1000 images.
Processed class 'Potato___healthy' with 1000 images.
Processed class 'Corn_(maize)___Northern_Leaf_Blight' with 1000 images.
Processed class 'Tomato___Early_blight' with 1000 images.
Processed class 'Tomato___Septoria_leaf_spot' with 1000 images.
Processed class 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot' with 1000 images.
Processed class 'Strawberry___Leaf_scorch' with 1000 images.
Processed class 'Peach___healthy' with 1000 images.
Processed class 'Apple___Apple_scab' with 1000 images.
Processed class 'Tomato___Tomato_Yellow_Leaf_Curl_Virus' with 1000 images.
Processed class 'Tomato___Bacterial_spot' with 1000 images.
Processed 

In [50]:
training_data = os.listdir("/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train")
test_data=glob('/kaggle/input/new-plant-diseases-dataset/test/test/*.JPG')
validation_data=os.listdir('/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid')

In [ ]:
def extract_features(image_path):
    
    image = cv2.imread(image_path)
  
    image = cv2.resize(image, (128, 128))
    
    hist = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist = cv2.normalize(hist, hist).flatten()

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    lbp = cv2.Laplacian(gray, cv2.CV_64F).var()  
    
    features = np.hstack([hist, lbp])
    return features


In [ ]:
training_images_path=dict()
for class_path in training_data:
    
    images_path=[]
    
    for class_image in os.listdir("/kaggle/working/reduced_dataset/train/"+class_path):
        images_path.append("/kaggle/working/reduced_dataset/train/"+class_path+"/"+class_image)
        
    training_images_path.update({class_path:images_path})
        


In [57]:
validation_images_path=dict()
for class_path in validation_data:
    images_path=[]
    
    for class_image in os.listdir("/kaggle/working/reduced_dataset/valid/"+class_path):
        images_path.append("/kaggle/working/reduced_dataset/valid/"+class_path+"/"+class_image)
    
    validation_images_path.update({class_path:images_path})


In [58]:
prepared_training_data=dict()
for class_element in training_images_path:
    features=[]
    for image_path in training_images_path.get(class_element):
        features.append(extract_features(image_path))
    prepared_training_data.update({class_element:features})

In [59]:
prepared_validation_data=dict()
for class_element in validation_images_path:
    features=[]
    for image_path in validation_images_path.get(class_element):
        
        features.append(extract_features(image_path))
    prepared_validation_data.update({class_element:features})

In [60]:
x=[]
y=[]
i=0
for class_name, feature_list in prepared_training_data.items():
    for features in feature_list:
        x.append(features)  # Add the feature vector
        y.append(i)
    i=i+1


In [61]:
clf=tree.DecisionTreeRegressor()

clf.fit(x,y)

DecisionTreeRegressor()

In [63]:
x_test=[]
y_test=[]
i=0
for class_name, feature_list in prepared_validation_data.items():
    for features in feature_list:
        x_test.append(features)  # Add the feature vector
        y_test.append(i)
    i=i+1
y_pred=clf.predict(x_test)

In [64]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.64      0.63      0.64       300
           1       0.90      0.93      0.92       300
           2       0.76      0.82      0.79       300
           3       0.76      0.71      0.74       300
           4       0.71      0.63      0.67       300
           5       0.80      0.72      0.76       300
           6       0.75      0.82      0.79       300
           7       0.74      0.75      0.75       300
           8       0.71      0.72      0.72       300
           9       0.61      0.57      0.59       300
          10       0.65      0.69      0.67       300
          11       0.84      0.78      0.81       300
          12       0.88      0.91      0.89       300
          13       0.59      0.58      0.58       300
          14       0.69      0.66      0.68       300
          15       0.71      0.66      0.68       300
          16       0.75      0.71      0.73       300
          17       0.71    

In [5]:
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(20, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=label_encoder.classes_, 
            yticklabels=label_encoder.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

NameError: name 'y_test' is not defined